In [1]:
import os
# 完全禁用GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # 减少日志输出

import numpy as np
import tensorflow as tf

# 确保TensorFlow使用CPU
tf.config.set_visible_devices([], 'GPU')

def load_and_predict():
    """使用CPU进行SavedModel推理"""
    try:
        print("使用CPU加载SavedModel...")
        
        # 加载SavedModel
        model = tf.saved_model.load("./saved_model/saved_model_dir")
        
        # 获取推理函数
        infer = model.signatures["serving_default"]
        
        # 检查输入签名
        print("模型输入签名:")
        for key, spec in infer.structured_input_signature[1].items():
            print(f"  {key}: {spec}")
        
        # 准备输入数据
        test_image = np.random.rand(1, 224, 224, 3).astype(np.float32)
        print(f"输入数据形状: {test_image.shape}")
        
        # 转换为tensor并指定设备
        with tf.device('/CPU:0'):
            input_tensor = tf.constant(test_image)
            
            # 进行预测
            predictions = infer(input_tensor)
        
        # 获取输出
        output_key = list(predictions.keys())[0]
        result = predictions[output_key].numpy()
        
        print("Raw prediction output:", result)
        print(f"输出形状: {result.shape}")
        
        if result.shape[-1] == 1:
            predicted_class = (result > 0.5).astype(int)
            print("Predicted class (0/1):", predicted_class)
        else:
            predicted_class = np.argmax(result, axis=-1)
            print("Predicted class index:", predicted_class)
            
        return result
            
    except Exception as e:
        print(f"预测失败: {e}")
        import traceback
        traceback.print_exc()
        return None

# 执行预测
result = load_and_predict()

2025-07-17 00:22:00.737439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752682920.747379   33461 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752682920.750439   33461 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752682920.758736   33461 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752682920.758768   33461 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752682920.758769   33461 computation_placer.cc:177] computation placer alr

使用CPU加载SavedModel...


2025-07-17 00:22:02.056372: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


模型输入签名:
  efficientnetb5_input: TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='efficientnetb5_input')
输入数据形状: (1, 224, 224, 3)
Raw prediction output: [[0.41770187]]
输出形状: (1, 1)
Predicted class (0/1): [[0]]
